# Reading Data

To do any data science, the first thing you're going to need is data. Let's start to look at a few common ways of reading data in Python. Along the way, we'll take a look at some language constructs that we'll use throughout the day and really start to get familiar with coding in Python.

### File handling in Python

First, let's look at file objects in Python. We get a file handle by using the **open** function.

In [2]:
csv_file = open("data/health_inspection_chi_sample.csv")

File objects are lazy **iterators** (here, *stream objects*). Lazy means that they only do things, in this case read data, when you ask them to. You can call **next** on iterator objects to explicitly get the next item.

In [3]:
line = next(csv_file)

In [4]:
print(line)

address,aka_name,city,dba_name,facility_type,inspection_date,inspection_id,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip



Here, the next item is a string which is the line in a file. In this case, it's the headers of the `csv` file.

The nice thing about **iterators** is that you can iterate through them with a for loop.

In [ ]:
for line in csv_file:
    pass

In [ ]:
print(line)

After consuming the file, we need to close the open file-handle.

In [ ]:
csv_file.close()

We can avoid that by using what's called a **context manager** in Python. This is a really nice abstraction. You use context managers via the **with** statement.

In [ ]:
with open("data/health_inspection_chi_sample.csv") as csv_file:
    for line in csv_file:
        pass

By using the `open` function as a context manager, we get an automatic call to close the open file when we exit the context (determined by the indentation level). When working with files non-interactively, you'll almost always want to use open as a context manager.

## Exercise

Write some code that iterates through the file `data/health_inspection_chi_sample.json` twice. Only call `open` once, however, then close the file. Can you find out, programatically, how many characters are in the file (hint: have a look at the `tell` method of the file object)?

In [ ]:
# Type your solution here

In [ ]:
%load solutions/read_json_twice.py

### Reading Data: CSVs

We peaked at this file above. We might already see some things we're going to have to clean up. Let's look at how we may do this. As always, Python has *batteries included*. The `csv` built-in module is great for basic data-munging needs.

In [5]:
import csv
from pprint import pprint

Let's open a csv file and use csv's `reader` function to read in the data.

In [7]:
csv_file = open("data/health_inspection_chi_sample.csv")

reader = csv.reader(csv_file)

`csv.reader` also returns a lazy iterator.

In [9]:
headers = next(reader)

In [10]:
pprint(headers)

['address',
 'aka_name',
 'city',
 'dba_name',
 'facility_type',
 'inspection_date',
 'inspection_id',
 'inspection_type',
 'latitude',
 'license_',
 'location',
 'longitude',
 'results',
 'risk',
 'state',
 'violations',
 'zip']


In [11]:
line = next(reader)

In [12]:
pprint(line)

['5255 W MADISON ST ',
 'RED SNAPPER FISH CHICKEN & PIZZA',
 'CHICAGO',
 'RED SNAPPER FISH CHICKEN & PIZZA',
 'Restaurant',
 '2016-09-26T00:00:00.000',
 '1965287',
 'Canvass',
 '41.880236543865834',
 '1991820.0',
 "{'type': 'Point', 'coordinates': [-87.757220392117, 41.880236543866]}",
 '-87.7572203921175',
 'Pass w/ Conditions',
 'Risk 1 (High)',
 'IL',
 '35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, '
 'SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: MUST CLEAN THE '
 'WALLS AT WALL BASE NEAR THE MIXER IN REAR OF PREMISES AND THE PREP AREA OF '
 'FOOD SPILLS AND CLEAN THE WALL VENT IN PREP AREA ,INSTRUCTED TO CLEAN AND '
 'MAINTAIN AREA | 33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSILS CLEAN, FREE '
 'OF ABRASIVE DETERGENTS - Comments: MUST CLEAN THE INTERIOR PANEL OF THE ICE '
 'MACHINE IN REAR OF PREMISES | 34. FLOORS: CONSTRUCTED PER CODE, CLEANED, '
 'GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED - Comments: '
 'MUST CLEAN 

The biggest difference in using `csv.reader` vs. iterating through the file is that it automatically splits the csv on commas and returns the line of the file split into a list.

You can control this behavior through a `Dialect` object. By default, `csv.reader` uses a Dialect object called "excel." Here let's look at the attributes of the excel dialect. Don't worry too much about the code used to achieve this. We'll look more at this later.

In [13]:
items = csv.excel.__dict__.items()

pprint({key: value for key, value in items if not key.startswith("_")})

{'delimiter': ',',
 'doublequote': True,
 'lineterminator': '\r\n',
 'quotechar': '"',
 'quoting': 0,
 'skipinitialspace': False}


You might also find `DictReader` to be useful. 

First, let's back up in the file to the beginning.

In [14]:
csv_file.seek(0)

0

In [ ]:
reader = csv.DictReader(csv_file)

In [15]:
pprint(next(reader))

['address',
 'aka_name',
 'city',
 'dba_name',
 'facility_type',
 'inspection_date',
 'inspection_id',
 'inspection_type',
 'latitude',
 'license_',
 'location',
 'longitude',
 'results',
 'risk',
 'state',
 'violations',
 'zip']


In [17]:
csv_file.close()

One last handy trick with the `csv` module is to use `Sniffer` to determine the csv dialect for you.

In [18]:
file_name = "data/health_inspection_chi_sample.csv"

with open(file_name) as csv_file:
    
    try:
        dialect = csv.Sniffer().sniff(csv_file.read(1024))
    except csv.Error as err:
        # log that this file format couldn't be deduced
        print(f"The format of {file_name} could not be detected.")
    else:
        csv_file.seek(0)
    
        dta = csv.reader(csv_file, dialect=dialect)

A big part of increased productivity is saving yourself some work later. The first rule of dealing with data is probably something like "The data is against you. Act accordingly."

We set this block of code up to be pretty paranoid. Software engineers call this **defensive programming**. It's a good habit to get into when you're doing any data science work in Python. 

There are three things to note here. First, is the use of `Sniffer` at all. If `file_name` is a standard csv format, we'll be able to read it.

The second is using a `try/except/else` block. In a `try/except` block, any exception that is raised will trigger the code in the except block. If no exception is raised, the *optional* else block will run.

Let's take a look at a toy example to fix ideas.

In [19]:
try:
    1/0
except:
    print("Something went wrong.")

Something went wrong.


In [31]:
try:
    1/0
except ZeroDivisionError as err:
    print("Something went wrong.")
    raise(err)

Something went wrong.


ZeroDivisionError: division by zero

In [32]:
try:
    1/0
except FileNotFoundError:
    print("This error isn't raised, so we're not here.")

ZeroDivisionError: division by zero

The final thing to note in the block above is the use of `print` to provide some information about what went wrong. Logging is another really good habit to get into, and print statements are the dead simplest way to log the behavior of your code.

In practice, you probably don't want to use `print`. You want to use the [logging](https://docs.python.org/3/library/logging.html) module, but we're not going to talk about best practices in logging anymore today.

### Reading Data: json

Perhaps the second most common file format after CSVs is the `JSON` format. JSON stands for JavaScript Object Notation. When reading data from an API, for example, you will often encounter json files.

Each line in the file `data/health_inspection_chi_sample.json` is a single json object that represents the same data above. 

In [33]:
!head -n 1 data/health_inspection_chi_sample.json

{"address":"5255 W MADISON ST ","aka_name":"RED SNAPPER FISH CHICKEN & PIZZA","city":"CHICAGO","dba_name":"RED SNAPPER FISH CHICKEN & PIZZA","facility_type":"Restaurant","inspection_date":"2016-09-26T00:00:00.000","inspection_id":1965287,"inspection_type":"Canvass","latitude":41.8802365439,"license_":1991820.0,"location":{"type":"Point","coordinates":[-87.7572203921,41.8802365439]},"longitude":-87.7572203921,"results":"Pass w\/ Conditions","risk":"Risk 1 (High)","state":"IL","violations":"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: MUST CLEAN THE WALLS AT WALL BASE NEAR THE MIXER IN REAR OF PREMISES AND THE PREP AREA OF FOOD SPILLS AND CLEAN THE WALL VENT IN PREP AREA ,INSTRUCTED TO CLEAN AND MAINTAIN AREA | 33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSILS CLEAN, FREE OF ABRASIVE DETERGENTS - Comments: MUST CLEAN THE INTERIOR PANEL OF THE ICE MACHINE IN REAR OF PREMISES | 34. FLOORS: CONSTRUCTED PE

We can use the `json` module to read and manipulate json data.

In [35]:
import json

Since each line is a json object here, we need to iterate over the file and parse each line. We use the `json.loads` function here for "load string." The similar function `json.load` takes a file-like object.

In [36]:
dta = []

with open("data/health_inspection_chi_sample.json") as json_file:
    for line in json_file:
        line = json.loads(line)
        dta.append(line)

pprint(dta[0])

{'address': '5255 W MADISON ST ',
 'aka_name': 'RED SNAPPER FISH CHICKEN & PIZZA',
 'city': 'CHICAGO',
 'dba_name': 'RED SNAPPER FISH CHICKEN & PIZZA',
 'facility_type': 'Restaurant',
 'inspection_date': '2016-09-26T00:00:00.000',
 'inspection_id': 1965287,
 'inspection_type': 'Canvass',
 'latitude': 41.8802365439,
 'license_': 1991820.0,
 'location': {'coordinates': [-87.7572203921, 41.8802365439], 'type': 'Point'},
 'longitude': -87.7572203921,
 'results': 'Pass w/ Conditions',
 'risk': 'Risk 1 (High)',
 'state': 'IL',
 'violations': '35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: '
               'GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - '
               'Comments: MUST CLEAN THE WALLS AT WALL BASE NEAR THE MIXER IN '
               'REAR OF PREMISES AND THE PREP AREA OF FOOD SPILLS AND CLEAN '
               'THE WALL VENT IN PREP AREA ,INSTRUCTED TO CLEAN AND MAINTAIN '
               'AREA | 33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSILS CLEAN

`json.loads` places each json object into a Python dictionary, helpfully filling in `None` for `null` for missing values and otherwise preserving types. It also, works recursively as we see in the `location` field.

## Aside: List Comprehensions

Let's take a look at another Pythonic concept, introduced a bit above, called a **list comprehension**. This is what's called *syntactic sugar*. It's a concise way to create a list.

In [37]:
[i for i in range(1, 6)]

[1, 2, 3, 4, 5]

Alternatively, we could have made this list by writing

In [38]:
result_list = []

for i in range(1, 6):
    result_list.append(i)

result_list

[1, 2, 3, 4, 5]

List comprehensions can contain logic.

In [ ]:
x = ['a', 'b', 'c', 'd', '_e', '_f']

In [ ]:
[i for i in x if not i.startswith('_')]

You can also use a an else clause. Notice the slightly different syntax.

In [ ]:
[i if not i.startswith('_') else 'skipped' for i in x]

List comprehensions can be nested, though it's usually best practices not to go overboard. They can quickly become difficult to read.

In [ ]:
matrix = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
]

Notice that we write this from outside to in.

In [ ]:
[element for row in matrix for element in row]

You can also create **dictionary comprehensions**.

In [ ]:
pairs = [
    ("first_initial", "J"), 
    ("last_name", "Doe"), 
    ("address", "1234 Main Street")
]

In [ ]:
{key: value for key, value in pairs}

## Exercise

Returning to the code that we introduced above, we can take further control over how a file with json objects is read in by using the `object_hook` argument. Say we wanted to remove the `location` field above. We don't need the `geoJSON` formatted information. We could do so with the `object_hook`. Write a function called `remove_entry` that removes the `'location'` field from each record in the `'data/health_inspection_chi_sample.json'` file.

Pass this function to the `object_hook` argument of `json.loads`. Be careful, `object_hook` will get called recursively on nested json objects.

In [ ]:
# Type your solution here

In [ ]:
%load solutions/object_hook_json.py

### Reading Data: pandas

Now let's take a look at using [pandas]() to read data.

#### Introducing Pandas

First, a few words of introduction for **pandas**. Pandas is a Python package providing fast, flexible, and expressive data structures designed to work with relational or labeled data. It is a high-level tool for doing practical, real world data analysis in Python.

You reach for pandas when you have:

* Tabular data with heterogeneously-typed columns
* Ordered and unordered (not necessarily fixed-frequency) time series data.
* Arbitrary matrix data with row and column labels

Almost any dataset can be converted to a pandas data structure for cleaning, transformation, and analysis.

Pandas is built on top of **numpy**, which we'll look a bit more at later.

First, let's import the pandas.

In [39]:
import pandas as pd

Let's set some display options.

In [40]:
pd.set_option("max.rows", 6)

Pandas has facilities for reading csv files and files containing JSON records (and other formats). We can use `read_csv` for csv files.

In [41]:
pd.read_csv("data/health_inspection_chi_sample.csv")

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_id,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip
0,5255 W MADISON ST,RED SNAPPER FISH CHICKEN & PIZZA,CHICAGO,RED SNAPPER FISH CHICKEN & PIZZA,Restaurant,2016-09-26T00:00:00.000,1965287,Canvass,41.880237,1991820.0,"{'type': 'Point', 'coordinates': [-87.75722039...",-87.757220,Pass w/ Conditions,Risk 1 (High),IL,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",60644.0
1,5958 W DIVERSEY AVE,TAQUERIA MORELOS,CHICAGO,TAQUERIA MORELOS,Restaurant,2014-02-06T00:00:00.000,1329698,Canvass,41.931250,2099479.0,"{'type': 'Point', 'coordinates': [-87.77590699...",-87.775907,Pass,Risk 1 (High),IL,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,60639.0
2,5400-5402 N CLARK ST,HAMBURGER MARY'S/MARY'S REC ROOM,CHICAGO,HAMBURGER MARY'S CHICAGO/MARY'S REC ROOM,Restaurant,2010-12-03T00:00:00.000,470787,SFP,41.979884,1933748.0,"{'type': 'Point', 'coordinates': [-87.66842948...",-87.668429,Fail,Risk 1 (High),IL,"6. HANDS WASHED AND CLEANED, GOOD HYGIENIC PRA...",60640.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,3251 E 92ND ST,VICTORY CENTRE OF SOUTH CHICAGO SLF,CHICAGO,VICTORY CENTRE OF SOUTH CHICAGO SLF,ASSISTED LIVING,2012-10-12T00:00:00.000,1296216,Canvass,41.728214,1968546.0,"{'type': 'Point', 'coordinates': [-87.54469279...",-87.544693,Pass,Risk 1 (High),IL,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",60617.0
998,4300 S UNION AVE,TUXEDO JUNCTION #2,CHICAGO,TUXEDO JUNCTION #2,Grocery Store,2011-07-14T00:00:00.000,585793,Canvass,41.816186,8974.0,"{'type': 'Point', 'coordinates': [-87.64345182...",-87.643452,Fail,Risk 2 (Medium),IL,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",60609.0
999,3820 W CHICAGO AVE,Bro-N-Laws,CHICAGO,Bro-N-Laws,Restaurant,2015-08-19T00:00:00.000,1566470,Complaint Re-Inspection,41.895513,2288968.0,"{'type': 'Point', 'coordinates': [-87.72216981...",-87.722170,Pass,Risk 1 (High),IL,45. FOOD HANDLER REQUIREMENTS MET - Comments: ...,60651.0


`read_csv` is one of the best/worst functions in all of Python. It's great because it does just about everything. It's terrible, because it does just about everything. Chances are if you have a special case that pandas `read_csv` will accomodate your needs. Go ahead and have a look at the `docstring`.


```python
pd.read_csv?
```

In [42]:
pd.read_csv?

`read_csv` returns a pandas DataFrame. We'll take a deeper dive into DataFrames next when we start to clean this data set.

The JSON counterpart to `read_csv` is `read_json`.

## Exercise

Use `pd.read_json` to read in the Chicago health inspections json sample in the `data` folder.

In [43]:
# Type your solution Here
pd.read_json("data/health_inspection_chi_sample.json")

ValueError: Trailing data

In [55]:
# %load solutions/read_json.py
import pandas as pd

pd.read_json(
    "data/health_inspection_chi_sample.json",
    orient="records",
    lines=True,
)


,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_id,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip
0,5255 W MADISON ST,RED SNAPPER FISH CHICKEN & PIZZA,CHICAGO,RED SNAPPER FISH CHICKEN & PIZZA,Restaurant,2016-09-26T00:00:00.000,1965287,Canvass,41.880237,1991820.0,"{'type': 'Point', 'coordinates': [-87.75722039...",-87.757220,Pass w/ Conditions,Risk 1 (High),IL,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",60644
1,5958 W DIVERSEY AVE,TAQUERIA MORELOS,CHICAGO,TAQUERIA MORELOS,Restaurant,2014-02-06T00:00:00.000,1329698,Canvass,41.931250,2099479.0,"{'type': 'Point', 'coordinates': [-87.77590699...",-87.775907,Pass,Risk 1 (High),IL,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,60639
2,5400-5402 N CLARK ST,HAMBURGER MARY'S/MARY'S REC ROOM,CHICAGO,HAMBURGER MARY'S CHICAGO/MARY'S REC ROOM,Restaurant,2010-12-03T00:00:00.000,470787,SFP,41.979884,1933748.0,"{'type': 'Point', 'coordinates': [-87.66842948...",-87.668429,Fail,Risk 1 (High),IL,"6. HANDS WASHED AND CLEANED, GOOD HYGIENIC PRA...",60640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,3251 E 92ND ST,VICTORY CENTRE OF SOUTH CHICAGO SLF,CHICAGO,VICTORY CENTRE OF SOUTH CHICAGO SLF,ASSISTED LIVING,2012-10-12T00:00:00.000,1296216,Canvass,41.728214,1968546.0,"{'type': 'Point', 'coordinates': [-87.54469279...",-87.544693,Pass,Risk 1 (High),IL,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",60617
998,4300 S UNION AVE,TUXEDO JUNCTION #2,CHICAGO,TUXEDO JUNCTION #2,Grocery Store,2011-07-14T00:00:00.000,585793,Canvass,41.816186,8974.0,"{'type': 'Point', 'coordinates': [-87.64345182...",-87.643452,Fail,Risk 2 (Medium),IL,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",60609
999,3820 W CHICAGO AVE,Bro-N-Laws,CHICAGO,Bro-N-Laws,Restaurant,2015-08-19T00:00:00.000,1566470,Complaint Re-Inspection,41.895513,2288968.0,"{'type': 'Point', 'coordinates': [-87.72216981...",-87.722170,Pass,Risk 1 (High),IL,45. FOOD HANDLER REQUIREMENTS MET - Comments: ...,60651


### Reading Data: web data

So far, we've seen some ways that we can read data from disk. As Data Scientists, we often need to go out and grab data from the Internet.

Generally Python is "batteries included" and reading data from the Internet is no exception, but there are some *great* packages out there. [requests](http://docs.python-requests.org/en/master/) is one of them for making HTTP requests.

Let's look at how we can use the [Chicago Data Portal](https://data.cityofchicago.org/) API to get this data in the first place. (I originally used San Francisco for this, but the data was just too clean to be terribly interesting.)

In [46]:
import requests

We use the requests library to perform a GET request to the API, passing an optional query string via `params` to limit the returned number of records. The parameters are documented as part of the [Socrata Open Data API](https://dev.socrata.com/consumers/getting-started.html) (SODA).

In [47]:
response = requests.get(
    "https://data.cityofchicago.org/resource/cwig-ma7x.json", 
    params="$limit=10"
)

Requests returns a [Reponse](http://docs.python-requests.org/en/master/api/#requests.Response) object with many helpful methods and attributes.

In [48]:
response

<Response [200]>

In [49]:
response.ok

True

In [56]:
dta = pd.read_json(response.content, orient='records')

We can use the `head` method to peak at the first 5 rows of data.

In [57]:
dta.head()

,:@computed_region_43wa_7qmu,:@computed_region_6mkv_f3dw,:@computed_region_awaf_s7ux,:@computed_region_bdys_3d7i,:@computed_region_vrxf_vc4k,address,aka_name,city,dba_name,facility_type,...,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip
0,38.0,21869,28.0,748,13.0,4042 W FOSTER AVE,JEWEL FOOD STORE #3442,CHICAGO,JEWEL FOOD STORE #3442,Grocery Store,...,Complaint,41.975624,1406,"{'type': 'Point', 'coordinates': [-87.72984456...",-87.729845,Pass w/ Conditions,Risk 1 (High),IL,40. PERSONAL CLEANLINESS - Comments: MUST REMO...,60630
1,21.0,21538,27.0,71,22.0,2902-2906 N CENTRAL PARK AVE,BRIGHT BEGINNINGS EARLY LEARNING CENTER LLC,CHICAGO,BRIGHT BEGINNINGS EARLY LEARNING CENTER LLC,Children's Services Facility,...,License Re-Inspection,41.933964,2363848,"{'type': 'Point', 'coordinates': [-87.71739089...",-87.717391,Pass,Risk 1 (High),IL,NaN,60618
2,NaN,16197,NaN,170,NaN,11601 W TOUHY AVE,FRONTERA TORTAS BY RICK BAYLESS (T1-B11),CHICAGO,FRONTERA TORTAS BY RICK BAYLESS,Restaurant,...,Canvass,42.008536,2363760,"{'type': 'Point', 'coordinates': [-87.91442843...",-87.914428,Pass,Risk 1 (High),IL,49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...,60666
3,NaN,16197,NaN,170,NaN,11601 W TOUHY AVE,WOLFGANG PUCK (T1 B7),CHICAGO,WOLFGANG PUCK T1,Restaurant,...,Canvass,42.008536,34157,"{'type': 'Point', 'coordinates': [-87.91442843...",-87.914428,Pass,Risk 1 (High),IL,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,60666
4,29.0,22532,34.0,782,12.0,5400 W DEVON AVE,STARBUCKS COFFEE #2479,CHICAGO,STARBUCKS COFFEE #2479,Restaurant,...,Canvass,41.997437,1095196,"{'type': 'Point', 'coordinates': [-87.76385919...",-87.763859,Pass w/ Conditions,Risk 2 (Medium),IL,39. CONTAMINATION PREVENTED DURING FOOD PREPAR...,60646


Of course, pandas can also load data directly from a URL, but I encourage you to reach for `requests` as often as you need it.

## Exercise

Try passing the URL above to `pd.read_json`. What happens?

In [71]:
# Type your solution here
import pandas as pd

dta_ex = pd.read_json(
    "https://data.cityofchicago.org/resource/cwig-ma7x.json",
    orient = "records"
)
dta_ex

,:@computed_region_43wa_7qmu,:@computed_region_6mkv_f3dw,:@computed_region_awaf_s7ux,:@computed_region_bdys_3d7i,:@computed_region_vrxf_vc4k,address,aka_name,city,dba_name,facility_type,...,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip
0,5.0,21853.0,3.0,267.0,10.0,6977-6981 N SHERIDAN RD,NIBBLES & NOSH/MIDNIGHT MAC & CHEESERIE/XO MAR...,CHICAGO,NIBBLES & NOSH/MIDNIGHT MAC & CHEESERIE/XO MAR...,NaN,...,License,42.009029,2658934,"{'type': 'Point', 'coordinates': [-87.66188729...",-87.661887,No Entry,Risk 1 (High),IL,NaN,60626.0
1,NaN,16197.0,NaN,170.0,NaN,11601 W TOUHY AVE,FRONTERA TORTAS BY RICK BAYLESS (T1-B11),CHICAGO,FRONTERA TORTAS BY RICK BAYLESS,Restaurant,...,Canvass,42.008536,2363760,"{'type': 'Point', 'coordinates': [-87.91442843...",-87.914428,Pass,Risk 1 (High),IL,49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...,60666.0
2,29.0,22532.0,34.0,782.0,12.0,5400 W DEVON AVE,STARBUCKS COFFEE #2479,CHICAGO,STARBUCKS COFFEE #2479,Restaurant,...,Canvass,41.997437,1095196,"{'type': 'Point', 'coordinates': [-87.76385919...",-87.763859,Pass w/ Conditions,Risk 2 (Medium),IL,39. CONTAMINATION PREVENTED DURING FOOD PREPAR...,60646.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,36.0,14914.0,48.0,786.0,29.0,225 S CANAL ST,GOLD COAST DOGS,CHICAGO,GOLD COAST DOGS,Restaurant,...,Canvass Re-Inspection,41.878531,32318,"{'type': 'Point', 'coordinates': [-87.63942745...",-87.639427,Pass w/ Conditions,Risk 2 (Medium),IL,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,60606.0
998,20.0,21538.0,2.0,214.0,16.0,4305 N KEDZIE AVE,TEZCAL RESTAURANT,CHICAGO,TEZCAL RESTAURANT,Restaurant,...,Canvass,41.959572,2626654,"{'type': 'Point', 'coordinates': [-87.70804923...",-87.708049,Pass w/ Conditions,Risk 1 (High),IL,23. PROPER DATE MARKING AND DISPOSITION - Comm...,60618.0
999,8.0,22248.0,49.0,640.0,55.0,3322 W 47TH ST,LA HACIENDITA TAQUERIA,CHICAGO,LA HACIENDITA TAQUERIA INC,Restaurant,...,Canvass,41.808186,1479601,"{'type': 'Point', 'coordinates': [-87.70743369...",-87.707434,Pass w/ Conditions,Risk 1 (High),IL,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",60632.0


In [66]:
# %load solutions/read_url_json.py
import pandas as pd

url = ('https://data.cityofchicago.org/'
              'resource/cwig-ma7x.json?$limit=5')
pd.read_json(url, orient='records')


,:@computed_region_43wa_7qmu,:@computed_region_6mkv_f3dw,:@computed_region_awaf_s7ux,:@computed_region_bdys_3d7i,:@computed_region_vrxf_vc4k,address,aka_name,city,dba_name,facility_type,...,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip
0,21.0,21538,27.0,71,22.0,2902-2906 N CENTRAL PARK AVE,BRIGHT BEGINNINGS EARLY LEARNING CENTER LLC,CHICAGO,BRIGHT BEGINNINGS EARLY LEARNING CENTER LLC,Children's Services Facility,...,License Re-Inspection,41.933964,2363848,"{'type': 'Point', 'coordinates': [-87.71739089...",-87.717391,Pass,Risk 1 (High),IL,NaN,60618
1,29.0,22532,34.0,782,12.0,5400 W DEVON AVE,STARBUCKS COFFEE #2479,CHICAGO,STARBUCKS COFFEE #2479,Restaurant,...,Canvass,41.997437,1095196,"{'type': 'Point', 'coordinates': [-87.76385919...",-87.763859,Pass w/ Conditions,Risk 2 (Medium),IL,39. CONTAMINATION PREVENTED DURING FOOD PREPAR...,60646
2,5.0,21853,3.0,267,10.0,6977-6981 N SHERIDAN RD,NIBBLES & NOSH/MIDNIGHT MAC & CHEESERIE/XO MAR...,CHICAGO,NIBBLES & NOSH/MIDNIGHT MAC & CHEESERIE/XO MAR...,NaN,...,License,42.009029,2658934,"{'type': 'Point', 'coordinates': [-87.66188729...",-87.661887,No Entry,Risk 1 (High),IL,NaN,60626
3,NaN,16197,NaN,170,NaN,11601 W TOUHY AVE,WOLFGANG PUCK (T1 B7),CHICAGO,WOLFGANG PUCK T1,Restaurant,...,Canvass,42.008536,34157,"{'type': 'Point', 'coordinates': [-87.91442843...",-87.914428,Pass,Risk 1 (High),IL,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,60666
4,NaN,16197,NaN,170,NaN,11601 W TOUHY AVE,FRONTERA TORTAS BY RICK BAYLESS (T1-B11),CHICAGO,FRONTERA TORTAS BY RICK BAYLESS,Restaurant,...,Canvass,42.008536,2363760,"{'type': 'Point', 'coordinates': [-87.91442843...",-87.914428,Pass,Risk 1 (High),IL,49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...,60666


In [67]:
pd.read_json('https://data.cityofchicago.org/'
              'resource/cwig-ma7x.json?$limit=5',
             orient='records')

,:@computed_region_43wa_7qmu,:@computed_region_6mkv_f3dw,:@computed_region_awaf_s7ux,:@computed_region_bdys_3d7i,:@computed_region_vrxf_vc4k,address,aka_name,city,dba_name,facility_type,...,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip
0,21.0,21538,27.0,71,22.0,2902-2906 N CENTRAL PARK AVE,BRIGHT BEGINNINGS EARLY LEARNING CENTER LLC,CHICAGO,BRIGHT BEGINNINGS EARLY LEARNING CENTER LLC,Children's Services Facility,...,License Re-Inspection,41.933964,2363848,"{'type': 'Point', 'coordinates': [-87.71739089...",-87.717391,Pass,Risk 1 (High),IL,NaN,60618
1,29.0,22532,34.0,782,12.0,5400 W DEVON AVE,STARBUCKS COFFEE #2479,CHICAGO,STARBUCKS COFFEE #2479,Restaurant,...,Canvass,41.997437,1095196,"{'type': 'Point', 'coordinates': [-87.76385919...",-87.763859,Pass w/ Conditions,Risk 2 (Medium),IL,39. CONTAMINATION PREVENTED DURING FOOD PREPAR...,60646
2,5.0,21853,3.0,267,10.0,6977-6981 N SHERIDAN RD,NIBBLES & NOSH/MIDNIGHT MAC & CHEESERIE/XO MAR...,CHICAGO,NIBBLES & NOSH/MIDNIGHT MAC & CHEESERIE/XO MAR...,NaN,...,License,42.009029,2658934,"{'type': 'Point', 'coordinates': [-87.66188729...",-87.661887,No Entry,Risk 1 (High),IL,NaN,60626
3,NaN,16197,NaN,170,NaN,11601 W TOUHY AVE,WOLFGANG PUCK (T1 B7),CHICAGO,WOLFGANG PUCK T1,Restaurant,...,Canvass,42.008536,34157,"{'type': 'Point', 'coordinates': [-87.91442843...",-87.914428,Pass,Risk 1 (High),IL,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,60666
4,NaN,16197,NaN,170,NaN,11601 W TOUHY AVE,FRONTERA TORTAS BY RICK BAYLESS (T1-B11),CHICAGO,FRONTERA TORTAS BY RICK BAYLESS,Restaurant,...,Canvass,42.008536,2363760,"{'type': 'Point', 'coordinates': [-87.91442843...",-87.914428,Pass,Risk 1 (High),IL,49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...,60666


Notice how you can split a string across lines. This can be a very handy tip for improving readability, by splitting a string and putting it in parentheses, we preserve a single string.

In [ ]:
("super "
 "long "
 "string "
 "split "
 "across "
 "lines")

#### Pandas DataReader

In addition to the core I/O functionality in pandas, there is also the [pandas-datareader](https://pandas-datareader.readthedocs.io/en/latest/) project. This package provides programmatic access to data sets from

* Yahoo! Finance (deprecated)
* Google Finance
* Enigma
* Quandl
* FRED
* Fama/French
* World Bank
* OECD
* Eurostat
* EDGAR Index (deprecated)
* TSP Fund Data
* Nasdaq Trader Symbol Definitions
* Morningstar
* Etc.

#### Further Resources

Sometimes we need to be resourceful in order to get data. Knowing how to scrape the web can really come in handy.

We're not going to go into details today, but you'll likely find libraries like [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/), [lxml](http://lxml.de/), and [mechanize](https://mechanize.readthedocs.io/en/latest/) to be helpful. 

There's also a `read_html` function in pandas that will quickly scrape HTML tables for you and put them into a DataFrame. 